In [1]:
import os
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

Using TensorFlow backend.


In [2]:
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
from keras.models import load_model

In [4]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D

In [5]:
def save_labels(arr, filename):
    pd_array = pd.DataFrame(arr)
    pd_array.index.names = ["Id"]
    pd_array.columns = ["Prediction"]
    pd_array.to_csv(filename)

def load_labels(filename):
    return pd.read_csv(filename, index_col=0).values.ravel()

W drugiej części projektu będe używał autocoderów do uczenia konwolucyjnej sieci neuronowej. Polega to na tym, że możemy wykorzystać dane ze zbioru uczącego bez etykiet. By to zrobić na początku uczymy sieć która ma zadanie aproksymować idetyczność to znaczy dla każdego obrazka z Cifar10 ma zwrócić jak najbardziej podobny obrazek. Po tym wykorzystujemy część nauczonych przez tę sieć warst do stworzenia nowej sieci, której zadaniem będzie już klasyfikacja obrazków. Dzięki temu pierwsze warstwy sieci neuronowej nie będą zainicjalizowane losowo. 

Będe wykorzystywał podobnne sieci jak w pierwszym projekcie. Nie będe zmieniał hiperparametrów autoencodera ponieważ jego uczenie zajmuje dość dużo czasu

matemaczninie autoencoder można przedstawić ze złożniem funkcji $d:X\rightarrow Y$ dekodującej oraz funkcji $e:Y\rightarrow X$. 

Zwykle przestrzeń Y powina być mniej wymiarowa niż X.

Będziemy uczyć funkcje d oraz e za pomocą jednego z algorytmów gradentowych tak aby $e \circ d$ aproksymowało identyczność

Mój autoencoder tranformuje wyjściowe obrazki w następujący sposób gdzie w nawiasach podane są wymiary macierzy

(wejście)=(3,32,32)->(32,32,32)->(16,16,32)->(16,16,1) tą macierz będzie zwracała funkcja kodująca

->(16,16,1)->(1,32,32)->(32,32,32)->(3,32,32)=(wyjście)


Potem przetranformuje moje dane $X_{train\_small}$ za pomocą nauczonej funkcji d i będe uczyć sieć, której wejście będą dane ze zbioru $d(X_{train\_small})$

Potem porównam je z sieciami mające na wejście $X_{train\_small}$

Modele wykorzystjące encoder zaczeły się overfitować po 40 epokach

Modele działające na obrazkach zaczeły się ovefitować dużo szybciej.

Nie ma jednak dość dużej różnicy w accussy modelów działających na zwykłych danych a danych przetworzonych przez encoder


Być może powiniem dłużej uczyć autocoder albo użyć innej architektury ale jest bardzo czasochłone (szczególnie wybranie odpowiedniej architektury autocodera)

In [9]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
X_train_small=np.load("X_train_small.npy")
y_train_small = load_labels("y_train_small.csv")
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train_small = X_train_small.astype('float32')
X_train /= np.max(X_train) 
X_train_small/= np.max(X_train_small) 
X_test /= np.max(X_test) 
X_train=X_train.reshape(-1,32,32,3)
X_train_small=X_train_small.reshape(-1,32,32,3)
Y_train_small = np_utils.to_categorical(y_train_small, 10) 

In [37]:
inp = Input(shape=(32, 32, 3))

x = Convolution2D(32, (3, 3), padding='same', activation='relu')(inp)
x = Convolution2D(4, (3, 3), padding='same', activation='relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)


x = Convolution2D(4, (3, 3), activation='relu', padding='same', data_format="channels_last")(encoded)
x=UpSampling2D((2, 2), data_format="channels_last")(x)
x = Convolution2D(32, (3, 3), activation='relu', padding='same', data_format="channels_last")(x)
decoded = Convolution2D(3, (3, 3), activation='sigmoid', padding='same', data_format="channels_last")(x)

autoencoder = Model(inp, decoded)
encoder = Model(inp, encoded)

In [40]:
def model_generator(conv_layer1_size,conv_layer2_size,layer3_size,kernel_size):
    drop_prob_1=0.23
    drop_prob_2=0.5
    pool_size=2
    inp = Input(shape=(16, 16,4))
    conv_1 = Convolution2D(conv_layer1_size, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    conv_2 = Convolution2D(conv_layer2_size, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    hidden = Dense(layer3_size, activation='relu')(flat)
    drop_3 = Dropout(drop_prob_2)(hidden)
    out = Dense(10, activation='softmax')(drop_3)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 
    return model

In [38]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(X_train,X_train,epochs=10,validation_split=0.05)

Train on 47500 samples, validate on 2500 samples
Epoch 1/10
47500/47500 [==============================] - 268s - loss: 0.6023 - val_loss: 0.5880
Epoch 2/10
47500/47500 [==============================] - 244s - loss: 0.5815 - val_loss: 0.5770
Epoch 3/10
47500/47500 [==============================] - 252s - loss: 0.5757 - val_loss: 0.5782
Epoch 4/10
47500/47500 [==============================] - 245s - loss: 0.5738 - val_loss: 0.5767
Epoch 5/10
47500/47500 [==============================] - 250s - loss: 0.5724 - val_loss: 0.5718
Epoch 6/10
47500/47500 [==============================] - 256s - loss: 0.5715 - val_loss: 0.5712
Epoch 7/10
47500/47500 [==============================] - 262s - loss: 0.5706 - val_loss: 0.5701
Epoch 8/10
47500/47500 [==============================] - 239s - loss: 0.5700 - val_loss: 0.5693
Epoch 9/10
47500/47500 [==============================] - 239s - loss: 0.5693 - val_loss: 0.5688
Epoch 10/10
47500/47500 [==============================] - 254s - loss: 0.5689

In [43]:
X_small_tran=encoder.predict(X_train_small)

In [55]:
#Generujemy 3 lowych modeli o różnych hiperparametrach
Rand_model_table=[model_generator(
    np.random.randint(20,71), #liczba filtrów w pierszej warstwie konwolucji 
    np.random.randint(20,71), #liczba filtrów w drugiej warstwie konwolucji
    np.random.randint(100,500), #liczba filtrów w trzeciej warstwie (zwykła warstwa)
    np.random.randint(2,4), #rozmiar filtrów dla obu warst konwolucji 
)   for i in range(3)]

In [56]:
#trenujemy modele w tablicy i zwracamy ich wynik dla zbioru walidującego
def models_scory(Model_table,X_train,y_train,X_ewal,y_ewal,nazwa_pliku,epochs=10):
    i=0
    for model in Model_table:
        i=i+1       
        model.fit(X_train,y_train,batch_size=32, epochs=epochs)
        model.save(nazwa_pliku+str(i)+'.h5') #zapisuje model
    return np.array([(model,model.evaluate(X_ewal,y_ewal, verbose=1)[1]) for model in Model_table]) 
#zwracamy pary (model,accuracy na zbiorze validującym)  

In [59]:
tab=models_scory(Rand_model_table,X_small_tran[0:4500],Y_train_small[0:4500],X_small_tran[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 5s - loss: 2.2132 - acc: 0.1700     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 2.0039 - acc: 0.2656     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.8841 - acc: 0.3111     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.8159 - acc: 0.3464     
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 1.7492 - acc: 0.3691     - ETA: 0s - loss: 1.7456 - acc
Epoch 1/5
4500/4500 [==============================] - 4s - loss: 2.2286 - acc: 0.1593     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 2.0473 - acc: 0.2489     
Epoch 3/5
4500/4500 [==============================] - 3s - loss: 1.9428 - acc: 0.2849     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.8561 - acc: 0.3309     
Epoch 5/5
4500/4500 [==============================] - 3s - loss: 1.8299 - acc: 0.3371     
Epoch 1/5
4500/4500 [=============================

In [60]:
tab

array([[<keras.engine.training.Model object at 0x000001A4C6420BA8>,
        0.36799999976158143],
        0.36199999976158143],
        0.34599999976158141]], dtype=object)

In [61]:
tab=models_scory(Rand_model_table,X_small_tran[0:4500],Y_train_small[0:4500],X_small_tran[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 4s - loss: 1.7028 - acc: 0.3804     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 1.6679 - acc: 0.4007     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.6375 - acc: 0.4144     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.5980 - acc: 0.4320     
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 1.5604 - acc: 0.4276     - ETA: 2s - loss: 1.5742 - ac - ETA: 1s - l
Epoch 1/5
4500/4500 [==============================] - 4s - loss: 1.7644 - acc: 0.3589     
Epoch 2/5
4500/4500 [==============================] - 3s - loss: 1.7312 - acc: 0.3753     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.7191 - acc: 0.3782     
Epoch 4/5
4500/4500 [==============================] - 3s - loss: 1.6788 - acc: 0.3969     
Epoch 5/5
4500/4500 [==============================] - 3s - loss: 1.6495 - acc: 0.4138     
Epoch 1/5
4500/4500 [================

In [62]:
tab

array([[<keras.engine.training.Model object at 0x000001A4C6420BA8>,
        0.41000000047683716],
        0.43199999976158143],
        0.39000000000000001]], dtype=object)

In [63]:
tab=models_scory(Rand_model_table,X_small_tran[0:4500],Y_train_small[0:4500],X_small_tran[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 4s - loss: 1.5508 - acc: 0.4482     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 1.5145 - acc: 0.4598     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.4911 - acc: 0.4578     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.4680 - acc: 0.4798     
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 1.4379 - acc: 0.4889     
Epoch 1/5
4500/4500 [==============================] - 3s - loss: 1.6455 - acc: 0.4084     
Epoch 2/5
4500/4500 [==============================] - 3s - loss: 1.6149 - acc: 0.4213     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.5931 - acc: 0.4280     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.5745 - acc: 0.4293     - ETA: 0s - loss: 1.5822 - acc:
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 1.5494 - acc: 0.4471     
Epoch 1/5
4500/4500 [============================

In [64]:
tab

array([[<keras.engine.training.Model object at 0x000001A4C6420BA8>,
        0.44000000047683718],
        0.42199999952316286],
        0.41199999976158141]], dtype=object)

In [65]:
tab=models_scory(Rand_model_table,X_small_tran[0:4500],Y_train_small[0:4500],X_small_tran[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 4s - loss: 1.4298 - acc: 0.4844     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 1.3586 - acc: 0.5149     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.3496 - acc: 0.5093     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.3453 - acc: 0.5202     
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 1.3113 - acc: 0.5256     
Epoch 1/5
4500/4500 [==============================] - 3s - loss: 1.5328 - acc: 0.4491     - ET
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 1.5066 - acc: 0.4536     
Epoch 3/5
4500/4500 [==============================] - 3s - loss: 1.4978 - acc: 0.4518     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.4761 - acc: 0.4671     
Epoch 5/5
4500/4500 [==============================] - 3s - loss: 1.4362 - acc: 0.4813     
Epoch 1/5
4500/4500 [==============================] - 3s - loss: 1.5455 - a

In [71]:
tab=models_scory(Rand_model_table,X_small_tran[0:4500],Y_train_small[0:4500],X_small_tran[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 4s - loss: 1.1777 - acc: 0.5718     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 1.1471 - acc: 0.5864     
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.1184 - acc: 0.6056     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.0877 - acc: 0.6064     
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 1.0887 - acc: 0.6180     
Epoch 1/5
4500/4500 [==============================] - 3s - loss: 1.3591 - acc: 0.5076     
Epoch 2/5
4500/4500 [==============================] - 3s - loss: 1.3272 - acc: 0.5336     
Epoch 3/5
4500/4500 [==============================] - 3s - loss: 1.3114 - acc: 0.5320     
Epoch 4/5
4500/4500 [==============================] - 3s - loss: 1.3084 - acc: 0.5438     
Epoch 5/5
4500/4500 [==============================] - 3s - loss: 1.3133 - acc: 0.5347     
Epoch 1/5
4500/4500 [==============================] - 2s - loss: 1.4028 - acc: 

In [74]:
tab=models_scory(Rand_model_table,X_small_tran[0:4500],Y_train_small[0:4500],X_small_tran[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 4s - loss: 1.0576 - acc: 0.6253     
Epoch 2/5
4500/4500 [==============================] - 4s - loss: 1.0534 - acc: 0.6182     - ETA: 0s - loss: 1.0497 - ac
Epoch 3/5
4500/4500 [==============================] - 4s - loss: 1.0239 - acc: 0.6378     
Epoch 4/5
4500/4500 [==============================] - 4s - loss: 1.0126 - acc: 0.6320     
Epoch 5/5
4500/4500 [==============================] - 4s - loss: 0.9860 - acc: 0.6540     
Epoch 1/5
4500/4500 [==============================] - 3s - loss: 1.2859 - acc: 0.5407     
Epoch 2/5
4500/4500 [==============================] - 3s - loss: 1.2862 - acc: 0.5400     
Epoch 3/5
4500/4500 [==============================] - 3s - loss: 1.2464 - acc: 0.5516     
Epoch 4/5
4500/4500 [==============================] - 3s - loss: 1.2396 - acc: 0.5467     
Epoch 5/5
4500/4500 [==============================] - 3s - loss: 1.2131 - acc: 0.5724     
Epoch 1/5
4500/4500 [==============================

In [97]:
tab #usunołem wcześniejsze wyniky ale lepsze

array([[<keras.engine.training.Model object at 0x000001A4C6420BA8>,
        0.44599999976158144],
        0.46999999999999997],
        0.44199999976158144]], dtype=object)

In [99]:
model=tab[2][0]

In [102]:
Y_test=model.predict(encoder.predict(X_test.reshape(-1,32,32,3)))

In [104]:
y_test_predict=np.array([np.argmax(x) for x in Y_test])

In [105]:
y_test_predict

array([3, 8, 9, ..., 5, 1, 7], dtype=int64)

In [106]:
save_labels(y_test_predict, "y_small_pred.csv")

Modele porównanie z modelami bez transormacji

In [79]:
def model_generator2(conv_layer1_size,conv_layer2_size,layer3_size,kernel_size):
    drop_prob_1=0.23
    drop_prob_2=0.5
    pool_size=2
    inp = Input(shape=(32, 32,3))
    conv_1 = Convolution2D(conv_layer1_size, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    conv_2 = Convolution2D(conv_layer2_size, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    flat = Flatten()(drop_2)
    hidden = Dense(layer3_size, activation='relu')(flat)
    drop_3 = Dropout(drop_prob_2)(hidden)
    out = Dense(10, activation='softmax')(drop_3)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 
    return model

In [86]:
#Generujemy 3 lowych modeli o różnych hiperparametrach
Rand_model_table2=[model_generator2(
    np.random.randint(20,71), #liczba filtrów w pierszej warstwie konwolucji 
    np.random.randint(20,71), #liczba filtrów w drugiej warstwie konwolucji
    np.random.randint(100,500), #liczba filtrów w trzeciej warstwie (zwykła warstwa)
    np.random.randint(2,4), #rozmiar filtrów dla obu warst konwolucji 
)   for i in range(3)]

In [87]:
tab2=models_scory(Rand_model_table2,X_train_small[0:4500],Y_train_small[0:4500],X_train_small[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=5)

Epoch 1/5
4500/4500 [==============================] - 8s - loss: 2.2604 - acc: 0.1478     
Epoch 2/5
4500/4500 [==============================] - 7s - loss: 2.0572 - acc: 0.2462     
Epoch 3/5
4500/4500 [==============================] - 8s - loss: 1.8821 - acc: 0.3184     
Epoch 4/5
4500/4500 [==============================] - 10s - loss: 1.7601 - acc: 0.3644    
Epoch 5/5
4500/4500 [==============================] - 9s - loss: 1.6908 - acc: 0.3978     
Epoch 1/5
4500/4500 [==============================] - 14s - loss: 2.1930 - acc: 0.1851    
Epoch 2/5
4500/4500 [==============================] - 14s - loss: 1.8453 - acc: 0.3280    
Epoch 3/5
4500/4500 [==============================] - 14s - loss: 1.6879 - acc: 0.3887    - ETA: 2
Epoch 4/5
4500/4500 [==============================] - 14s - loss: 1.6147 - acc: 0.4249    
Epoch 5/5
4500/4500 [==============================] - 14s - loss: 1.5232 - acc: 0.4498    
Epoch 1/5
4500/4500 [==============================] - 7s - loss: 2.1881

In [88]:
tab2 #model 2 

array([[<keras.engine.training.Model object at 0x000001A4CDD32C18>,
        0.42199999976158142],
        0.4840000002384186],
        0.43199999952316281]], dtype=object)

In [89]:
tab3=models_scory(Rand_model_table2,X_train_small[0:4500],Y_train_small[0:4500],X_train_small[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=1)

Epoch 1/1
4500/4500 [==============================] - 7s - loss: 1.6437 - acc: 0.3936     
Epoch 1/1
4500/4500 [==============================] - 12s - loss: 1.4560 - acc: 0.4762    
Epoch 1/1
448/500 [=========================>....] - ETA: 0s

In [90]:
#modele dużo szybciej zaczynają się overfitować ale nie ma znacznej różniczy w accurasy modeli operujących na zwykłych
#obrazkach z cifar-10 a ich transformacjami dokonanych przez encoder
tab3

array([[<keras.engine.training.Model object at 0x000001A4CDD32C18>,
        0.42399999952316286],
        0.44600000023841857],
        0.47200000023841859]], dtype=object)

In [91]:
tab3=models_scory(Rand_model_table2,X_train_small[0:4500],Y_train_small[0:4500],X_train_small[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=1)

Epoch 1/1
4500/4500 [==============================] - 7s - loss: 1.6030 - acc: 0.4229     
Epoch 1/1
4500/4500 [==============================] - 12s - loss: 1.4267 - acc: 0.4902    
Epoch 1/1
416/500 [=======================>......] - ETA: 0s

In [92]:
tab3

array([[<keras.engine.training.Model object at 0x000001A4CDD32C18>,
        0.43000000023841856],
        0.50200000023841862],
        0.46000000023841858]], dtype=object)

In [93]:
tab3=models_scory(Rand_model_table2,X_train_small[0:4500],Y_train_small[0:4500],X_train_small[4500:5000],Y_train_small[4500:5000],"p2.1.",epochs=1)

Epoch 1/1
4500/4500 [==============================] - 7s - loss: 1.5649 - acc: 0.4371     
Epoch 1/1
4500/4500 [==============================] - 13s - loss: 1.3460 - acc: 0.5229    
Epoch 1/1
416/500 [=======================>......] - ETA: 0s

In [94]:
tab3

array([[<keras.engine.training.Model object at 0x000001A4CDD32C18>,
        0.4740000002384186],
        0.49600000023841856],
        0.47399999976158141]], dtype=object)